In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import matplotlib.pyplot as plt

if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    print('----------- Float tensor set --------------')
else:
    print('---------------------- No CUDA -----------------')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name = "BAAI/bge-large-en-v1.5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/torch/__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)


----------- Float tensor set --------------


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [4]:
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, 

In [4]:
targetText = 'magic is real'
print(tokenizer(targetText, return_tensors='pt'))
outputs = model(**tokenizer(targetText, return_tensors='pt'), output_hidden_states=True)
target_embedding = outputs.pooler_output[0]
print(target_embedding)

{'input_ids': tensor([[ 101, 3894, 2003, 2613,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
tensor([-0.9450, -0.8097, -0.8362,  ...,  0.4859,  0.9817, -0.8716],
       grad_fn=<SelectBackward0>)


In [ ]:
for i in range(len(outputs.hidden_states)):
    print(outputs.hidden_states[i])

In [18]:
outputs.pooler_output[0]

tensor([-0.9450, -0.8097, -0.8362,  ...,  0.4859,  0.9817, -0.8716],
       grad_fn=<SelectBackward0>)

In [19]:
model.pooler(outputs.hidden_states[-1])[0]

tensor([-0.9450, -0.8097, -0.8362,  ...,  0.4859,  0.9817, -0.8716],
       grad_fn=<SelectBackward0>)

In [7]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch.nn.functional as F

def cosine_similarity(embedding1, embedding2):
  # Normalize the embeddings
  embedding1_normalized = F.normalize(embedding1, p=2, dim=0)
  embedding2_normalized = F.normalize(embedding2, p=2, dim=0)

  # Calculate the dot product
  dot_product = torch.dot(embedding1_normalized, embedding2_normalized)

  return dot_product.item()

text1 = 'magic is real'
text2 = 'magic are real'
print(tokenizer(text1, return_tensors='pt'))
print(tokenizer(text2, return_tensors='pt'))
cosine_similarity(model(**tokenizer(text1, return_tensors='pt'), output_hidden_states=True).pooler_output[0], model(**tokenizer(text2, return_tensors='pt'), output_hidden_states=True).pooler_output[0])

{'input_ids': tensor([[ 101, 3894, 2003, 2613,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
{'input_ids': tensor([[ 101, 3894, 2024, 2613,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}


0.9960455894470215

In [28]:
cosineSimilarities = []

for i in range(len(outputs.hidden_states)):
    hiddenStateSimilarity = []
    embeddingI = model.pooler(outputs.hidden_states[i])[0]
    for j in range(len(outputs.hidden_states)):
        embeddingJ = model.pooler(outputs.hidden_states[j])[0]
        hiddenStateSimilarity.append(round(cosine_similarity(embeddingI, embeddingJ), 2))
    cosineSimilarities.append(hiddenStateSimilarity)

In [32]:
for i in range(len(cosineSimilarities)):
    print(f'{cosineSimilarities[i]}')

[1.0, 0.85, 0.84, 0.84, 0.85, 0.78, 0.59, 0.35, -0.0, -0.06, 0.01, 0.17, -0.01, 0.07, -0.08, 0.08, 0.1, -0.15, -0.17, -0.03, -0.01, 0.1, 0.11, 0.19, 0.16]
[0.85, 1.0, 1.0, 0.99, 0.99, 0.93, 0.7, 0.36, -0.05, -0.09, 0.05, 0.29, 0.07, 0.18, -0.01, 0.24, 0.26, -0.12, -0.2, -0.01, 0.01, 0.16, 0.17, 0.26, 0.27]
[0.84, 1.0, 1.0, 1.0, 0.99, 0.93, 0.71, 0.36, -0.07, -0.11, 0.04, 0.29, 0.06, 0.18, -0.01, 0.26, 0.29, -0.1, -0.19, 0.01, 0.04, 0.19, 0.2, 0.29, 0.3]
[0.84, 0.99, 1.0, 1.0, 1.0, 0.93, 0.7, 0.35, -0.08, -0.11, 0.03, 0.28, 0.05, 0.15, -0.03, 0.25, 0.28, -0.1, -0.18, 0.02, 0.04, 0.2, 0.21, 0.3, 0.3]
[0.85, 0.99, 0.99, 1.0, 1.0, 0.95, 0.74, 0.38, -0.09, -0.13, 0.02, 0.28, 0.05, 0.15, -0.04, 0.26, 0.29, -0.09, -0.18, 0.04, 0.06, 0.22, 0.23, 0.33, 0.33]
[0.78, 0.93, 0.93, 0.93, 0.95, 1.0, 0.91, 0.58, -0.04, -0.1, 0.08, 0.39, 0.12, 0.2, -0.02, 0.32, 0.37, -0.08, -0.19, 0.08, 0.1, 0.31, 0.31, 0.42, 0.45]
[0.59, 0.7, 0.71, 0.7, 0.74, 0.91, 1.0, 0.79, 0.07, -0.0, 0.18, 0.49, 0.21, 0.25, 0.01, 